In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import numpy as np
import pandas as pd
import torch
import evaluate
import matplotlib.pyplot as plt

from tensorflow.python.summary.summary_iterator import summary_iterator
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from scipy.special import expit
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.python.summary.summary_iterator import summary_iterator
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
np.random.seed(42)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/УИР/Интенциональности.csv")
def replace_wrong(s):
  if s == 'Воволечь в диалог':
    return 'Вовлечь в диалог'
  return s

df['label'] = df['label'].apply(replace_wrong)
df.to_csv('/content/drive/MyDrive/УИР/intents.csv', index=None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/УИР/intents.csv')
df.head()

,text,label
0,Что ты думаешь о последних событиях? Хотелось ...,Вовлечь в диалог
1,"Мне интересно, как ты видишь эту ситуацию. Мож...",Вовлечь в диалог
2,Давай обсудим наши планы на следующий месяц. Ч...,Вовлечь в диалог
3,"Я заметил, что ты умеешь решать эту проблему. ...",Вовлечь в диалог
4,"Послушай, у меня возникло несколько идей по ул...",Вовлечь в диалог


In [ ]:
labels = df.label.unique()
id = range(23)

labels2id = dict(zip(labels, id))
id2labels = dict(zip(id, labels))

In [ ]:
def split_data(df, label):
  df['label'] = df['label'].apply(lambda x: 1 if x == label else 0)
  true_labels = df[df['label'] == 1]
  false_labels = df[df['label'] == 0].sample(len(true_labels)*2, random_state=42)
  data = pd.concat([true_labels, false_labels], ignore_index=True)
  return train_test_split(data['text'].values, data['label'].values, test_size=0.25, random_state=42, stratify=data['label'].values)


In [ ]:
class TorchSet(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
metrics = {
    'label':[],
    'accuracy': [],
    'f1': [],
    'roc_auc': [],
    'recall': [],
    'precision': []
}
done = []
for label in labels:
  model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
  tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
  X_train, X_test, y_train, y_test = split_data(df.copy(), label)
  train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
  test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)
  train_dataset = TorchSet(train_encodings, y_train)
  test_dataset = TorchSet(test_encodings, y_test)
  training_args = TrainingArguments(
      output_dir=f'/content/drive/MyDrive/УИР/intent_models/{label}',
      num_train_epochs=7,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=64,
      warmup_steps=500,
      weight_decay=0.01,
      evaluation_strategy="steps",
      eval_steps=5,
      logging_dir=f'/content/drive/MyDrive/УИР/intent_logs/{label}',
      logging_steps=5,
      load_best_model_at_end=True
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
  )

  trainer.train()
  trainer.save_model(f'/content/drive/MyDrive/УИР/intent_models/{label}')
  done.append(f"/content/drive/MyDrive/УИР/intent_models/{label}")

  accuracy = evaluate.load("accuracy")
  f1 = evaluate.load("f1")
  roc_auc = evaluate.load("roc_auc")
  recall = evaluate.load('recall')
  precision = evaluate.load("precision")

  predictions = trainer.predict(test_dataset)
  preds = np.argmax(predictions.predictions, axis=-1)

  metrics['label'].append(label)
  metrics['accuracy'].append(list(accuracy.compute(predictions=preds, references=predictions.label_ids).values()))
  metrics['f1'].append(list(f1.compute(predictions=preds, references=predictions.label_ids).values()))
  metrics['roc_auc'].append(list(roc_auc.compute(prediction_scores=preds, references=predictions.label_ids).values()))
  metrics['recall'].append(list(recall.compute(predictions=preds, references=predictions.label_ids).values()))
  metrics['precision'].append(list(precision.compute(predictions=preds, references=predictions.label_ids).values()))

  train_loss = []
  eval_loss = []
  step = 0

  for e in trainer.state.log_history:
    if 'loss' in e.keys():
      train_loss.append(e['loss'])
    if 'eval_loss' in e.keys():
      eval_loss.append(e['eval_loss'])
    step = max(step, e['step'])

  steps = np.arange(5, step+1, 5)
  plt.plot(steps, train_loss, '-b', label='train loss')
  plt.plot(steps, eval_loss, '-r', label='test loss')

  plt.xlabel("step")
  plt.ylabel("loss")

  plt.legend()

  plt.savefig(f'/content/drive/MyDrive/УИР/Plots/{label.replace(" ", "_")}.png')
  plt.clf()



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
5,0.661200,0.677312
10,0.683200,0.672595
15,0.676800,0.664552
20,0.667600,0.653289
25,0.673100,0.639149
30,0.624300,0.619061
35,0.620100,0.591157
40,0.582700,0.562035
45,0.588500,0.529965
50,0.557200,0.489101


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
5,0.697300,0.685070
10,0.684300,0.682962
15,0.683800,0.678466
20,0.670600,0.671719
25,0.674700,0.664609
30,0.658300,0.656671
35,0.653000,0.649073
40,0.645300,0.642977
45,0.630700,0.637817
50,0.649500,0.633711


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
5,0.688200,0.688077
10,0.685200,0.684145
15,0.685400,0.677947
20,0.672000,0.669977
25,0.663700,0.661204
30,0.665700,0.653210
35,0.616600,0.643372
40,0.673800,0.634743
45,0.632300,0.629498
50,0.623200,0.625643


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
5,0.687200,0.688425
10,0.692800,0.684533
15,0.686200,0.678948
20,0.674200,0.671824
25,0.671000,0.663125
30,0.672400,0.654469
35,0.633700,0.645307
40,0.642300,0.636156
45,0.653000,0.629227
50,0.620100,0.619392


<Figure size 640x480 with 0 Axes>

In [ ]:
def to_int(x):
  if type(x)==list:
    return int(x)
  return x
met = pd.DataFrame(metrics).apply(lambda x:x[0] if type(x)==list else x)

In [ ]:
met.accuracy = met.accuracy.apply(lambda x:x[0] if type(x)==list else x)
met.f1 = met.f1.apply(lambda x:x[0] if type(x)==list else x)
met.roc_auc = met.roc_auc.apply(lambda x:x[0] if type(x)==list else x)
met.recall = met.recall.apply(lambda x:x[0] if type(x)==list else x)
met.precision = met.precision.apply(lambda x:x[0] if type(x)==list else x)
met.to_csv('/content/drive/MyDrive/УИР/intent_models/metrics.csv', index=None)